# Especializando dados com o DBT

O DBT (Data Build Tool) é uma ferramenta de orquestração de dados open source criada para ajudar equipes de dados a transformar, gerenciar e monitorar fluxos de dados de maneira mais eficiente. Com o dbt, é possível realizar tarefas como extrair, transformar e carregar dados, além de criar e gerenciar modelos de dados e tabelas.

## 1. Instalando

Abra o terminal do seu computador e execute o seguinte comando:

In [ ]:
!pip instal dbt-core

Como vamos utilizar o BigQuery como banco de dados, temos que instalar o plugin dbt-bigquery

In [ ]:
!pip install dbt-bigquery

Para certificar-se de ter o dbt Core instalado, verifique a versão usando o comando:

In [3]:
!dbt --version

Core:
  - installed: 1.4.5
  - latest:    1.4.5 - Up to date!

Plugins:
  - bigquery: 1.4.3 - Up to date!


## 2. Criando um projeto

Vamos aprender a usar uma série de comandos no Terminal para criar um projeto. 

O dbt Core possui um comando ``init`` que ajuda a estruturar um projeto dbt.

Para criar seu projeto dbt:

2.1. Escolha uma pasta de sua preferência e inicie um projeto em um diretório vazio, usando o comando:

dbt init

Ele irá pedir o nome do projeto. Digite "shop" (sem as aspas)

Como alternativa, você também pode passar o nome do projeto diretamente:

In [ ]:
dbt init shop

Em seguida, o dbt fará algumas perguntas para ajudá-lo a configurar seu projeto. As perguntas que o dbt faz podem variar um pouco dependendo da versão do dbt e das opções que você escolheu.

Algumas das perguntas que o dbt pode fazer incluem:



- *__Which database do you want to use?__*
    
  O dbt pode trabalhar com vários tipos de banco de dados, incluindo PostgreSQL, Redshift, BigQuery e Snowflake (veja a lista completa em: https://docs.getdbt.com/docs/supported-data-platforms). 
  
  Como vamos usar o Big Query, podemos escolher [1]

Agora, o dbt fará algumas perguntas específicas para configurar nosso projeto com o BigQuery. 

- __*Desired authentication method option (enter a number)*__

Essa pergunta é feita para permitir que você escolha como deseja autenticar sua conexão com o BigQuery.

O dbt suporta vários métodos de autenticação com o BigQuery, como autenticação baseada em chave de API, autenticação com credenciais de serviço do Google Cloud Platform (GCP) ou autenticação com OAuth. Dependendo do método de autenticação escolhido, o dbt precisará de diferentes informações de autenticação para se conectar com o BigQuery.

O dbt fornece uma lista numerada das opções de autenticação suportadas. Iremos optar por [2] service_account.

Após selecionar o método de autenticação, o dbt fará algumas perguntas adicionais para configurar a conexão com o BigQuery. 
- **keyfile**: Como escolhemos a opção de autenticação baseada em credenciais de serviço do GCP, aqui é onde devemos fornecer o caminho para o arquivo JSON com as credenciais de serviço.
- **project**: Aqui você deve informar o nome do projeto no GCP que contém o BigQuery dataset que deseja acessar.
- **dataset**: Informe o nome do dataset no BigQuery que você deseja usar para o seu projeto dbt.
- **Desired location option (enter a number)**: Informe a região do BigQuery em que o dataset está localizado. Por exemplo, us-central1, europe-west1, etc.


Em seguida, o dbt init irá fazer mais algumas perguntas para personalizar o ambiente do projeto.

- **threads (1 or more)**: quantas threads você deseja usar ao executar tarefas do dbt em paralelo. Threads são como fluxos de trabalho separados que podem ser executados simultaneamente. O dbt usa threads para paralelizar a execução de tarefas, como a compilação de modelos ou a execução de consultas. O número de threads que você escolher pode depender de vários fatores, como o tamanho do seu conjunto de dados e a capacidade de processamento do seu hardware. Você pode optar por escolher o número padrão sugerido pelo dbt ou ajustar o número de threads com base em suas necessidades específicas.


- **job_execution_timeout_seconds [300]**: permite definir um limite de tempo para a execução das tarefas do dbt. Isso é útil para garantir que as tarefas não sejam executadas indefinidamente e evita que o dbt fique preso em uma tarefa que nunca será concluída. O valor padrão para o tempo limite é de 300 segundos (ou 5 minutos), mas você pode optar por aumentar ou diminuir esse valor com base nas necessidades do seu projeto.

Responda às perguntas conforme a seguir:

In [ ]:
Enter a name for your project (letters, digits, underscore): shop
Which database would you like to use?
[1] bigquery

(Don't see the one you want? https://docs.getdbt.com/docs/available-adapters)

Enter a number: 1
[1] oauth
[2] service_account
Desired authentication method option (enter a number): 2
keyfile (/path/to/bigquery/keyfile.json): 
project (GCP project id): 
dataset (the name of your dbt dataset): 
threads (1 or more): 1
job_execution_timeout_seconds [300]: 300
[1] US
[2] EU
Desired location option (enter a number): 1
Profile proj written to C:\Users\alans\.dbt\profiles.yml using target's profile_template.yml and your supplied values. Run 'dbt debug' to validate the connection.

Your new dbt project "shop" was created!

For more information on how to configure the profiles.yml file,
please consult the dbt documentation here:

  https://docs.getdbt.com/docs/configure-your-profile

One more thing:

Need help? Don't hesitate to reach out to us via GitHub issues or on Slack:

  https://community.getdbt.com/

Happy modeling!

Depois de responder a todas essas perguntas, o dbt irá gerar um arquivo no diretório ``~/.dbt/`` chamado ``profiles.yml`` que contém as informações de conexão e autenticação necessárias para se conectar ao BigQuery.

2.2. Navegue até o diretório do seu projeto:

cd shop

2.3. Use um editor de código como o VSCode para abrir o diretório do projeto que você criou nas etapas anteriores. 

Para confirmar que você está no lugar certo, o diretório deve conter pastas e arquivos .sql e .yml gerados pelo comando ``init``.

2.4. Abra o arquivo *dbt_project.yml*, que deve estar localizado no diretório raiz do seu projeto, confira os seguintes valores e atualize-os, se necessário:

In [ ]:
name: dbt_shop # Altere do padrão, `my_new_project`

...

profile: dbt_shop # Altere do nome de perfil padrão, `default`

...

models:
  dbt_shop: # Obrigatoriamente deve corresponder ao valor de `name:`
...

Com essa configuração concluída, você poderá começar a usar o dbt para transformar seus dados e construir modelos no BigQuery.

## 3. Conectando ao BigQuery

Quando desenvolvemos localmente, o dbt se conecta ao seu data warehouse usando um perfil, que é um arquivo yaml com todos os detalhes de conexão com seu data warehouse.

1. Abra um arquivo chamado ``profiles.yml`` no diretório ``~/.dbt/``.

2. Mova seu arquivo de chave do BigQuery para este diretório.

3. Copie o seguinte código e cole no novo arquivo ``profiles.yml``. Certifique-se de atualizar os valores onde está comentado.

In [ ]:
dbt_shop: # isso precisa corresponder ao perfil em seu arquivo dbt_project.yml
  target: dev
  outputs:
    dev:
      type: bigquery
      method: service-account
      keyfile: # Caminho completo para o seu arquivo de chave
      project: # ID do seu projeto, exemplo dbt-shop
      dataset: # dbt_your_name, por exemplo dbt_alan
      threads: 1
      timeout_seconds: 300
      location: US
      priority: interactive

4. Execute o comando ``debug`` para confirmar que você se conectou com sucesso:

In [ ]:
dbt debug

Se tudo der certo, você deve obter uma resposta como a seguir:

In [ ]:
Configuration:
  profiles.yml file [OK found and valid]
  dbt_project.yml file [OK found and valid]

Required dependencies:
 - git [OK found]

Connection:
  method: service-account
  database: dbt-shop
  schema: dbt_alan
  location: US
  priority: interactive
  timeout_seconds: 300
  maximum_bytes_billed: None
  execution_project: dbt-shop
  job_retry_deadline_seconds: None
  job_retries: 1
  job_creation_timeout_seconds: None
  job_execution_timeout_seconds: 300
  gcs_bucket: None
  Connection test: [OK connection ok]

All checks passed!

## 4. Inserindo dados

Iremos utilizar o dataset Brazilian E-Commerce Public Dataset by Olist (https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce) um conjunto de dados públicos de comércio eletrônico brasileiro disponibilizado pela empresa Olist. O dataset inclui informações de pedidos, produtos, clientes, vendedores e pagamentos de mais de 100.000 pedidos realizados entre 2016 e 2018 em diferentes categorias de produtos.

## 5. Construindo um modelo 

Imagine que você precise gerar um relatório diário todas as manhãs antes das 10h para descobrir o número de pedidos por dia e representar o resultado em um dashboard (Looker ou Tableau, por exemplo).

Para isso, teria que criar uma tabela daily_order_count na IU do BigQuery para servir de entrada para o Looker ou Tableau com o seguinte esquema: 

create table 'dbt_shop.daily_order_count' (shipping_date date, num_orders int64)

Depois, todos os dias você terá que executar esta consulta para inserir valores na tabela daily_order_count:



In [ ]:
insert into `shiba-ecommerce.shiba_dataset.daily_order_count`
SELECT
   date(shipping_limit_date) as shipping_date,
   count(order_id) as num_orders
FROM 
   `shiba-ecommerce.shiba_dataset.shiba_order_items`
where 
   date(shipping_limit_date)='2018-06-13'
group by 1

Editar e executar esse .sql para será demorado e sujeito a erros.

Agora, em vez de executar manualmente a partir da IU do BigQuery, vamos criar modelos dbt (como arquivos .sql) e deixar que o dbt cuide do resto. 

Com o dbt instalado e configurado, podemos começar a usá-lo para preparar os dados e construir tabelas e visões especializadas. Para isso, precisamos criar arquivos SQL que definem como as transformações serão realizadas.

1. Crie um arquivo daily_order_count.sql em seu diretório *seu-projeto/models/daily_order_count.sql*. 
__Atenção__: O nome do arquivo sql deve ser exatamente igual ao nome da tabela no BigQuery, que é daily_order_count.

2. Copie a seguinte consulta em seu _daily_order_count.sql_

select
   date(shipping_limit_date) as shipping_date, 
   count(order_id) as num_orders 
FROM 
   `ecommerce.order_items` 
where 
   date(shipping_limit_date)='{{ var("ingestion_date") }}' 
group by 1

__Pergunta__: Qual a diferença entre o SQL que você executou agora comparando com o anterior?

__Resposta__: Removemos a instrução insert e mudamos os valores de shipping_limit_date para um arquivo .variable '{{ var("ingestion_date") }}'

O dbt fornece um mecanismo de variáveis para fornecer dados aos modelos para compilação. 

Nesse caso, as variáveis podem ser usadas para evitar que valores fiquem permanentes (hardcoded) na consulta. 

Para usar uma variável no modelo daily_order_count, use a função {{ var('...') }}

Acabamos de criar um **modelo**.

Modelo é um dos principais componentes de um projeto dbt. 

Cada modelo é declarado como um ``select`` e definido em um arquivo .sql. 

O nome do arquivo é usado como o nome do modelo. 

Voltando ao exemplo, o _daily_order_count.sql_ é um modelo.

## 6. Execute seu projeto DBT

Agora vamos usar ``dbt run`` para inserir esses valores na tabela ``daily_order_count``. 

Execute os seguintes comandos, um por um.

``dbt run --models shop.daily_order_count --vars "{"ingestion_date": "2018-06-29"}"``

``dbt run --models shop.daily_order_count --vars "{"ingestion_date": "2018-06-12"}"``

Execute duas vezes:

``dbt run --models shop.daily_order_count --vars "{"ingestion_date": "2018-06-13"}"``

## 7. Desafio

1. Crie uma tabela _orders_by_year_ a partir de _orders_ que agrupa a quantidade de pedidos por ano, considerando a coluna _order_approved_at_

2. Crie uma visão que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

3. Crie uma nova visão chamada _customers_by_region_ que utiliza os dados da tabela _customers_with_region_. A visão deve realizar uma agregação por região e contar o número de clientes em cada uma delas.

4. Crie uma nova tabela chamada _customers_with_region_ que utiliza os dados da tabela customers como base e adiciona uma nova coluna region que é calculada a partir da coluna customer_state. 
 
 **Dica**: utilize uma expressão CASE.
 
 **Exemplo**:
 
 CASE
        WHEN state IN ('SP', 'RJ') THEN 'Sudeste'
        
 CASE
        WHEN state IN ('CE', 'PI', 'MA', 'BA') THEN 'Nordeste'

5. Crie um novo modelo no diretório models/ com o nome *items_costs.sql* e defina uma consulta SQL que selecione algumas colunas da tabela order_items e calcule o preço total de cada item.

6. Crie um novo modelo no diretório models/ com o nome _orders_costs.sql_ e defina uma consulta SQL que agregue os dados da tabela orders e da tabela order_items para calcular o preço total de cada pedido.

## Testes no DBT

Quando você executa o comando dbt init para criar um novo projeto, o dbt pode gerar alguns testes prontos para garantir que o seu projeto esteja configurado corretamente e que o dbt esteja funcionando. 

Esses testes podem ser encontrados no arquivo models/schema.yml do seu projeto.

Os testes do dbt são escritos usando a sintaxe do jinja, uma linguagem de template que permite escrever código SQL dinâmico.

Por exemplo, o teste ``not_null_my_first_dbt_model_id`` é gerado automaticamente pelo ``dbt init`` para verificar se a coluna id na tabela my_first_dbt_model é definida como NOT NULL. Esse teste pode ser encontrado no diretório tests/schema_test.yml após a criação do novo projeto pelo dbt init.

Aqui está um exemplo do arquivo schema.yml que contém o teste not_null_my_first_dbt_model_id:

In [ ]:
version: 2

schema-tests:
  - name: not_null_my_first_dbt_model_id
    description: Verify that the `id` column in `my_first_dbt_model` is defined as NOT NULL
    models: my_first_dbt_model
    columns:
      - name: id
        tests:
          - not_null


Este teste verifica a coluna id para garantir que ela não é nula. Se a coluna id não for definida como NOT NULL, o teste falhará e você receberá uma mensagem de erro informando que o teste falhou e qual a causa do problema.

### Adicionando testes

Você pode editar os arquivos para personalizar os testes ou adicionar seus próprios testes. 


Adicionar testes a um projeto ajuda a validar se seus modelos estão funcionando corretamente e se as consultas SQL estão retornando os resultados esperados. Os testes também podem incluir verificações de consistência de dados, como verificar se uma tabela tem a coluna correta ou se o tipo de dados de uma coluna é o esperado.

Para adicionar testes ao seu projeto, altere o arquivo models/schema.yml

O dbt oferece quatro testes genéricos: **unique**, **not_null**, **accepted_values**, e **relationships**. 

Adicione o seguinte conteúdo ao arquivo models/schema.yml

In [ ]:
- name: customers
    columns:
      - name: customer_id
        tests:
          - unique
          - not_null

In [ ]:
- name: orders
    columns:
      - name: order_id
        tests:
          - unique
          - not_null
      - name: status
        tests:
          - accepted_values:
              values: ['placed', 'shipped', 'completed', 'return_pending', 'returned']
      - name: customer_id
        tests:
          - not_null
          - relationships:
              to: ref('customers')
              field: customer_id

In [15]:
#Vamos criar um script de teste sql que verifique se alguma das linhas do cliente foi duplicada ou perdida. 
#Se a consulta retornar um ou mais registros, os testes falharão.

version: 2

models:
  - name: customer_orders
    columns:
      - name: customer_id
        tests:
          - not_null
      - name: order_id
        tests:
          - unique
      - name: order_status
        tests:
          - accepted_values:
              values: ['delivered', 'invoiced', 'shipped', 'processing', 'canceled', 'unavailable']

Para executar os testes, você pode usar o comando ``dbt test``.

Quando você executa o ``dbt test``, o dbt itera pelos arquivos YAML e cria uma consulta para cada teste. 

Cada consulta retornará o número de registros que falharam no teste. Se esse número for 0, o teste foi bem-sucedido.

Execute dbt teste confirme se todos os seus testes foram aprovados.



In [ ]:
!dbt test